# Raghukanth & Iyengar (2007)

In which the GMPE of Raghukanth & Iyengar (2007) is implemented. First coefficient tables are reprocessed for cut & paste in to the .py source code. Then, key figures in the original paper are reproduced for validation. Finally, test vectors are produced for automatic code verification using unittest.

In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
import os
import warnings
import importlib
import pandas as pd
import numpy as np

import toolbox as tb
%autoreload 2
import gmpe_tools as gt

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from IPython.display import display

from openquake.hazardlib import gsim

from openquake.hazardlib.gsim.raghukanth_iyengar_2007 \
    import (RaghukanthIyengar2007, 
            RaghukanthIyengar2007KoynaWarna, 
            RaghukanthIyengar2007Southern, 
            RaghukanthIyengar2007WesternCentral)

In [ ]:
gmpe_short = 'RAIY07'
gmpes = [
    RaghukanthIyengar2007(),
    RaghukanthIyengar2007KoynaWarna(), 
    RaghukanthIyengar2007Southern(),
    RaghukanthIyengar2007WesternCentral(),
]
regions_short = [
    'PI',
    'KW',
    'SI',
    'WC',
]
regions_long = [
    'Peninsular India',
    'Koyna-Warna',
    'Southern',
    'Western-Central',
]

In [ ]:
bedrocks_csv = [
    'Table 3.csv',
    'Table 2a.csv', 
    'Table 2b.csv', 
    'Table 2c.csv', 
]
dfs_bedrock = [pd.DataFrame.from_csv('Tables/%s' % name) 
               for name in bedrocks_csv]
bedrocks_txt = [name.replace('.csv','.txt') for name in bedrocks_csv]
for df_bedrock, bedrock_txt in zip(dfs_bedrock, bedrocks_txt):
    with open('Tables/%s' % bedrock_txt,'w') as f:
        f.write(df_bedrock.to_string())

In [ ]:
sites_csv = 'Tables/Table 5.csv'
sites_txt = sites_csv.replace('.csv','.txt')
df_sites = pd.DataFrame.from_csv(sites_csv, header=[0,1])
for site_class in df_sites.columns.levels[0]:
    site_txt = sites_csv.replace('.csv','%s.txt' % site_class.lower())
    with open(site_txt,'w') as f:
        f.write(df_sites[site_class].to_string(sparsify=False))

In [ ]:
coefficients = ['c%d' % i for i in range(1,5)]
multipliers = [1., 1., 10., 300.]
fig, axes = plt.subplots(len(dfs_bedrock), 1, figsize=(6,10), sharex=True)
fig.subplots_adjust(hspace=0.1)

for ax, df_bedrock, bedrock_csv, region_long \
        in zip(axes, dfs_bedrock, bedrocks_csv, regions_long):
    row_label = '%s: %s' % (bedrock_csv.replace('.csv',''), region_long)
    ax.set_ylabel(row_label)
    for coeff, multiplier in zip(coefficients, multipliers):
        label = coeff
        if multiplier != 1:
            label = '%gx %s' % (multiplier, label)
        ax.semilogx(df_bedrock.index, multiplier*df_bedrock[coeff], 
                    marker='x', label=label)
    ax.grid(which='both')
    ax.set_ylim((-4, 4))
    ax.set_xlim((0.01, 4))
    
ax.set_xlabel('Period $T$ [s]')
axes[0].legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('Bedrock_coefficients.pdf', dpi=300, bbox_inches='tight')
plt.savefig('Bedrock_coefficients.png', dpi=300, bbox_inches='tight')

In [ ]:
df_site_coeffs = df_sites.loc[:,(slice(None), slice('a1','a2'))]
df_site_coeffs.plot(marker='x', logx=True)

plt.ylabel('Table 5: Site Class Coefficients')
plt.xlabel('Period $T$ [s]')
plt.grid(which='both')
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('Site_coefficients.pdf', dpi=300, bbox_inches='tight')
plt.savefig('Site_coefficients.png', dpi=300, bbox_inches='tight')

In [ ]:
df_sigmas = df_sites.loc[:,(slice(None), slice('sigma','sigma'))]
df_sigmas.columns = df_sigmas.columns.droplevel(1)
df_sigmas.is_copy = False
df_sigmas.loc[:,'bedrock'] = df_bedrock.loc[:,'sigma']
df_sigmas.plot(marker='x', logx=True)

plt.ylabel('Standard error $\\sigma(\\ln \\epsilon)$')
plt.xlabel('Period $T$ [s]')
plt.grid(which='both')
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('Sigmas.pdf', dpi=300, bbox_inches='tight')

In [ ]:
%autoreload 2
for gmpe in gmpes[0:1]:
    print(type(gmpe).__name__)
    print('Supported tectonic region: %s' 
          % gmpe.DEFINED_FOR_TECTONIC_REGION_TYPE)
    print('Supported intensity measure types: %s' 
          % ', '.join([item.__name__ for item 
                       in gmpe.DEFINED_FOR_INTENSITY_MEASURE_TYPES]))
    print('Supported component: %s' 
          % gmpe.DEFINED_FOR_INTENSITY_MEASURE_COMPONENT)
    print('Supported standard deviations: %s' 
          % ', '.join([item for item              
                       in gmpe.DEFINED_FOR_STANDARD_DEVIATION_TYPES]))
    print('Required site parameters: %s' 
          % ', '.join([item for item in gmpe.REQUIRES_SITES_PARAMETERS]))
    print('Required rupture parameters: %s' 
          % ', '.join([item for item in gmpe.REQUIRES_RUPTURE_PARAMETERS]))
    print('Required distance parameters: %s' 
          % ', '.join([item for item in gmpe.REQUIRES_DISTANCES]))

In [ ]:
# generate data for bedrock motion Figure 3
mags = [6.5]
rakes = [0] # degrees
distances = [35., 100.] # km
vs30s = 4000. # m/s
im_types = sorted(gmpes[0].COEFFS_BEDROCK.sa_coeffs.keys())

df_means = [gt.compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types)[0]
            for gmpe in gmpes[1:]]

df_stddevs = [gt.compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types)[1]
            for gmpe in gmpes[1:]]

df_means = pd.concat(df_means, keys=regions_short[1:], 
                     names=['gmpe','test'])
df_means.reset_index(inplace=True)
df_means.drop(['test'], axis=1, inplace=True)

df_stddevs = pd.concat(df_stddevs, keys=regions_short[1:], 
                     names=['gmpe','test'])
df_stddevs.reset_index(inplace=True)
df_stddevs.drop(['test'], axis=1, inplace=True)

In [ ]:
# produce Figure 3
digitized_template = 'Digitized/M%g_R%gkm_%s.csv'
empty = True

numeric_cols = np.array([tb.is_numeric(item) for item in df_means.columns])
extra_cols = df_means.columns[~numeric_cols]
T = [float(item) for item in df_means.columns[numeric_cols]]

gmpe_keys = sorted(list(set(df_means['gmpe'])))
rhypos = sorted(list(set(df_means['dist_rhypo'])))
fig, ax = plt.subplots(1, 1)
for gmpe_key in gmpe_keys:
    for rhypo in rhypos:
                
        df_trace = df_means[
            (df_means['gmpe'] == gmpe_key) &
            (df_means['dist_rhypo'] == rhypo)]
        if df_trace.size == 0:
            print 'No data found for', trace_label
            continue
            
        series = df_trace.drop(extra_cols, axis=1).sort(axis=1).iloc[0,:]
            
        trace_label = '%s %g km' % (gmpe_key, rhypo)
        h = ax.loglog(series.index, series.values,
                      label=trace_label, alpha=0.5)

        digitized_file = digitized_template % (mags[0], rhypo, gmpe_key)
        if os.path.exists(digitized_file):
            data = np.genfromtxt(digitized_file, delimiter=',')
            data = data[data[:,0].argsort()]
            ax.plot(data[:,0], data[:,1],
                    color=h[0].get_color(), marker='x', 
                    linestyle='none', alpha=0.5)
            
            df_trace.is_copy = False
            
            df_trace[T] = np.interp(T, data[:,0], data[:,1]).round(5)
            if empty:
                df_interp = df_trace
                empty = False
            else:
                df_interp = pd.concat((df_interp, df_trace), ignore_index=True)
        else:
            print '%s not available' % digitized_file
                
ax.grid(which='both')
ax.set_xlim((min(T), max(T)))
ax.set_xlabel('Period [s]')
ax.set_ylabel('SA($\\xi$=%g%%) [g]' % im_types[0].damping)
ax.legend(loc='lower left', labelspacing=0.3, fontsize=10)
plt.savefig('Figure_3_computed.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# generate data for bedrock motion Figure 5
mags = [6.5]
rakes = [0] # degrees
distances = [35.] # km
vs30s = [4000., 2000., 1000., 500., 250.] # m/s
im_types = sorted(gmpes[0].COEFFS_BEDROCK.sa_coeffs.keys())

df_means = gt.compute_gmpe(gmpes[0], mags, rakes, distances, vs30s, im_types)[0]
df_means['gmpe'] = regions_short[0]

df_stddevs2 = gt.compute_gmpe(gmpes[0], mags, rakes, distances, vs30s, im_types)[1]
df_stddevs2['gmpe'] = regions_short[0]
df_stddevs = pd.concat((df_stddevs, df_stddevs2))
df_stddevs = gt.df_massage(df_stddevs)

In [ ]:
df_stddevs

In [ ]:
# produce Figure 5
digitized_template = 'Digitized/Class_%s.csv'

vs30s = sorted(list(set(df_means['site_vs30'])), reverse=True)
sctx = gsim.base.SitesContext()
sctx.vs30 = np.array(vs30s)
site_classes = gmpe.get_nehrp_classes(sctx)
site_classes[gmpe.is_bedrock(sctx)] = 'bedrock'

numeric_cols = np.array([tb.is_numeric(item) for item in df_means.columns])
extra_cols = df_means.columns[~numeric_cols]
T = [float(item) for item in df_means.columns[numeric_cols]]

fig, ax = plt.subplots(1, 1)
for vs30, site_class in zip(vs30s, site_classes):
                
    df_trace = df_means[df_means['site_vs30'] == vs30]
    if df_trace.size == 0:
        print 'No data found for', trace_label
        continue
        
    series = df_trace.drop(extra_cols, axis=1).sort(axis=1).iloc[0,:]

    h = ax.loglog(series.index, series.values,
                  label=site_class, alpha=0.5)

    digitized_file = digitized_template % site_class
    if os.path.exists(digitized_file):
        data = np.genfromtxt(digitized_file, delimiter=',')
        data = data[data[:,0].argsort()]
        ax.plot(data[:,0], data[:,1],
                color=h[0].get_color(), marker='x', 
                linestyle='none', alpha=0.5)

        df_trace.is_copy = False
        df_trace[T] = np.interp(T, data[:,0], data[:,1], right=np.nan).round(5)
        df_interp = pd.concat((df_interp, df_trace), ignore_index=True)
    else:
        print '%s not available' % digitized_file
                
#ax.autoscale(enable=True, tight=True)
ax.grid(which='both')
ax.set_xlim((min(T), max(T)))
ax.set_xlabel('Period [s]')
ax.set_ylabel('SA($\\xi$=%g%%) [g]' % im_types[0].damping)
ax.legend(loc='lower left', labelspacing=0.3, fontsize=10)
plt.savefig('Figure_5_computed.pdf', dpi=300, bbox_inches='tight')

In [ ]:
df_interp = gt.df_massage(df_interp)
df_interp

In [ ]:
# check that warning is raised for low VS30
rctx = gsim.base.RuptureContext()
sctx = gsim.base.SitesContext()
dctx = gsim.base.DistancesContext()

rctx.mag = np.array([6.])
rctx.rake = np.array([-90.])
dctx.rhypo = np.array([100.])
sctx.vs30 = np.array([10.])
uncertainty_types = list(gmpe.DEFINED_FOR_STANDARD_DEVIATION_TYPES)

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter('always')
    # Trigger a warning.
    mean, [stddev] = gmpe.get_mean_and_stddevs(
        sctx, rctx, dctx, im_types[0], uncertainty_types)
    # Verify some things
    assert len(w) == 1
    assert issubclass(w[-1].category, UserWarning)
    assert 'not supported' in str(w[-1].message).lower()
    assert np.all(np.isnan(mean))

In [ ]:
# produce result files for unittest
test_path = '/home/nick/src/python/GEM/oq-hazardlib/openquake/hazardlib/tests/gsim/data/%s/' \
    % gmpe_short

means_files = [os.path.join(test_path, '%s_%s_MEAN.csv' 
                            % (gmpe_short, region_short))
               for region_short in regions_short]
for means_file, region_short in zip(means_files, regions_short):
    indices = df_interp['gmpe'] == region_short
    df_subset = df_interp[indices].drop(['rup_rake','gmpe'], axis=1).copy()
    df_subset = gt.df_massage(df_subset)
    df_subset.to_csv(means_file, index=False)
    
stddev_files = [os.path.join(test_path, '%s_%s_STD_TOTAL.csv' 
                            % (gmpe_short, region_short))
               for region_short in regions_short]

for stddev_file, region_short in zip(stddev_files, regions_short):
    indices = df_stddevs['gmpe'] == region_short
    df_subset = df_stddevs[indices].drop(['rup_rake','gmpe'], axis=1).copy()
    df_subset = gt.df_massage(df_subset)
    df_subset.to_csv(stddev_file, index=False)

In [ ]:
# run through unittests
threshhold = 0.015

fig, axes = plt.subplots(len(gmpes), 1, figsize=(6,12), sharex=True)
plt.subplots_adjust(hspace=0.05)

for ax, gmpe, means_file in zip(axes, gmpes, means_files):
    gmpe_name = gmpe.__class__.__name__
    ax.add_artist(AnchoredText(gmpe_name, loc=1, frameon=False))

    df_ref = gt.df_massage(pd.read_csv(means_file))

    mags = np.sort(np.array(list(set(df_ref['rup_mag']))))
    rakes = 0
    distances = sorted(list(set(df_ref['dist_rhypo'])))
    vs30s = np.sort(np.array(list(set(df_ref['site_vs30']))))
    sa_cols = np.array([tb.is_numeric(item) for item in df_ref.columns])
    imt_cols = np.array([gt.is_imt(item) for item in df_ref.columns])
    im_types = [gt.get_imt(item) for item in df_ref.columns[imt_cols]]

    df_new = gt.compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types)[0]
    df_new = df_new[df_ref.columns]

    for vs30 in vs30s:
        for dist in distances:
            trace_label = 'rhypo=%g km, vs30=%g m/s' % (dist, vs30)

            df_trace = df_ref[
                (df_ref['dist_rhypo'] == dist) & 
                (df_ref['site_vs30'] == vs30)]
            if df_trace.size == 0:
                continue
            ref = df_trace[df_ref.columns[sa_cols]]
            
            df_trace = df_new[
                (df_new['dist_rhypo'] == dist) & 
                (df_new['site_vs30'] == vs30)]
            new = df_trace[df_ref.columns[sa_cols]]
            
            h = ax.loglog(new.columns.values, new.values.reshape((-1,)), 
                          label=trace_label, alpha=0.5)

            ax.loglog(ref.columns.values, ref.values.reshape((-1,)), 
                      linestyle='none', marker='x', 
                     color=h[0].get_color(), alpha=0.5)
            

    ne_stacked = (tb.df_compare(df_new, df_ref) > threshhold).stack()
    changed = ne_stacked[ne_stacked]
    changed.index.names = ['index', 'column']

    difference_locations = np.where(
        tb.df_compare(df_new, df_ref) > threshhold)
    changed_from = df_new.values[difference_locations]
    changed_to = df_ref.values[difference_locations]
    summary = pd.DataFrame({'from': changed_from, 
                            'percent': 100*(changed_from/changed_to - 1)}, 
                           index=changed.index)
    if summary.size > 0:
        print
        print gmpe.__class__.__name__
        display(summary)
                
for ax in axes:
    ax.grid(which='both')
    ax.set_xlim((0.01, 4))
    ax.set_ylim((0.003, 1))
    ax.set_ylabel('SA($\\xi$=%g%%) [m/s$^2$]' % (im_types[0].damping))
    ax.legend(loc='lower left', labelspacing=0, fontsize=10)
axes[-1].set_xlabel('Period [s]')

plt.savefig('Unit_Test_Summary.pdf', dpi=300, bbox_inches='tight')
plt.savefig('Unit_Test_Summary.png', dpi=300, bbox_inches='tight')